### SNL Records by Season
This notebook collects record by season.

In [1]:
import pandas as pd
import numpy as np
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.palettes import Spectral11
output_notebook()

Loading BokehJS ...

In [2]:
fs = pd.read_csv('./db/snl_season.csv', encoding="utf-8")
dfe = pd.read_csv('./db/snl_episode.csv', encoding="utf-8",parse_dates=['aired'])
dft = pd.read_csv('./db/snl_title.csv', encoding="utf-8")
dfa = pd.read_csv('./db/snl_actor.csv', encoding="utf-8")
dfat = pd.read_csv('./db/snl_actor_title.csv', encoding="utf-8")
dfr = pd.read_csv('./db/snl_rating.csv', encoding="utf-8")

In [3]:
dfer = pd.merge(dfe, dfr, on=['sid', 'eid'])
dfactors = pd.merge(pd.merge(dfat, dfer, on=['sid', 'eid']), dfa, on='aid')

In [4]:
df_sea_cat = pd.DataFrame(dfactors.groupby(['actorType','sid'])['aid'].count()).reset_index()
df_sea_cat.columns = ['Type', 'Season', 'Appearances']
for actorType in df_sea_cat.Type.unique():
    df_sea_cat[actorType] = 0
    df_sea_cat.loc[df_sea_cat.Type==actorType, actorType] = df_sea_cat['Appearances']
    
df_sea_cat = df_sea_cat.drop(['Type'],axis=1)
df_sea_cat = df_sea_cat.groupby('Season').sum()

In [5]:
df_sea_cat

,Appearances,cameo,cast,crew,filmed,guest,host,music,unknown
Season,,,,,,,,,
1,1480,14,841,320,32,44,176,53,0
2,1428,17,780,387,30,6,148,60,0
3,1239,71,739,251,5,5,124,44,0
4,1112,17,691,255,9,2,103,35,0
5,1117,24,737,184,4,3,114,51,0
6,716,21,491,85,5,0,79,35,0
7,862,23,594,91,6,12,89,47,0
8,837,22,604,37,9,11,116,38,0
9,880,18,623,50,4,7,139,39,0


In [6]:
TOOLS=['pan','zoom_in','zoom_out','undo','redo','reset','save']
p = figure(plot_width=800, plot_height=800, y_range=(-10,df_sea_cat['Appearances'].max()),x_range=(0,45), tools=TOOLS)
#r = p.multi_line(['Season','Season','Season', 'Season', 'Season','Season','Season', 'Season', 'Season'],
 #                ['Appearances','cameo','cast','crew','filmed','guest','host','music','unknown'],
#              line_width=4, source=df_sea_cat)
numlines=len(df_sea_cat.columns)
mypalette=Spectral11[0:numlines]

p.multi_line(xs=[df_sea_cat.index.values]*numlines,
                ys=[df_sea_cat[name].values for name in df_sea_cat],
                line_color=mypalette,
                line_width=3)
t = show(p, notebook_handle=True)

In [7]:
df_sea_cat_avg = df_sea_cat.copy()
df_sea_cat_avg['Episodes'] = dfe.groupby(['sid'])['eid'].count()

In [8]:
for column in df_sea_cat_avg.columns:
    if column != 'Episodes':
        df_sea_cat_avg[column] = df_sea_cat_avg[column] / df_sea_cat_avg['Episodes']

In [9]:
TOOLS=['pan','zoom_in','zoom_out','undo','redo','reset','save']
p = figure(plot_width=800, plot_height=800, y_range=(-10,df_sea_cat_avg['Appearances'].max()),x_range=(0,45), tools=TOOLS)

for column in df_sea_cat_avg.columns:
    p.line(df_sea_cat_avg.index,df_sea_cat_avg[column], legend=column, line_color=Spectral11[df_sea_cat_avg.columns.get_loc(column)], line_width=3)

t = show(p, notebook_handle=True)